## Step 1: Load Data

In [0]:
%sql
SHOW TABLES IN workspace.default;


database,tableName,isTemporary
default,customer_details,false
default,e_commerece_sales_data_2024,false
default,product_details,false


In [0]:
spark.version

'4.0.0'

In [0]:
spark.sql("USE default")

DataFrame[]

In [0]:
sales_raw = spark.table("workspace.default.e_commerece_sales_data_2024")
products_raw = spark.table("workspace.default.product_details")
customers_raw = spark.table("workspace.default.customer_details")

display(sales_raw.limit(5))
display(products_raw.limit(5))
display(customers_raw.limit(5))

user id,product id,Interaction type,Time stamp
1,4c69b61db1fc16e7013b43fc926e502d,purchase,10/10/2023 8:00
2,66d49bbed043f5be260fa9f7fbff5957,view,11/10/2023 8:00
3,2c55cae269aebf53838484b0d7dd931a,like,12/10/2023 8:00
4,18018b6bc416dab347b1b7db79994afa,view,13/10/2023 8:00
5,e04b990e95bf73bbe6a3fa09785d7cd0,like,14/10/2023 8:00


Uniqe Id,Product Name,Brand Name,Asin,Category,Upc Ean Code,List Price,Selling Price,Quantity,Model Number,About Product,Product Specification,Technical Details,Shipping Weight,Product Dimensions,Image,Variants,Sku,Product Url,Stock,Product Details,Dimensions,Color,Ingredients,Direction To Use,Is Amazon Seller,Size Quantity Variant,Product Description
4c69b61db1fc16e7013b43fc926e502d,"DB Longboards CoreFlex Crossbow 41"" Bamboo Fiberglass Longboard Complete",null,null,"Sports & Outdoors | Outdoor Recreation | Skates, Skateboards & Scooters | Skateboarding | Standard Skateboards & Longboards | Longboards",null,null,$237.68,null,null,"Make sure this fits by entering your model number. | RESPONSIVE FLEX: The Crossbow features a bamboo core encased in triaxial fiberglass and HD plastic for a responsive flex pattern that’s second to none. Pumping & carving have never been so satisfying! Flex 2 is recommended for people 120 to 170 pounds. | COREFLEX TECH: CoreFlex construction is water resistant, impact resistant, scratch resistant and has a flex like you won’t believe. These boards combine fiberglass, epoxy, HD plastic and bamboo to create a perfect blend of performance and strength. | INSPIRED BY THE NORTHWEST: Our founding ideal is chasing adventure & riding the best boards possible, inspired by the hills, waves, beaches & mountains all around our headquarters in the Northwest | BEST IN THE WORLD: DB was founded out of sheer love of longboarding with a mission to create the best custom longboards in the world, to do it sustainably, & to treat customers & employees like family | BEYOND COMPARE: Try our skateboards & accessories if you've tried similar products by Sector 9, Landyachtz, Arbor, Loaded, Globe, Orangatang, Hawgs, Powell-Peralta, Blood Orange, Caliber or Gullwing",Shipping Weight: 10.7 pounds (View shipping rates and policies)|ASIN: B07KMVJJK7| #474 in Longboards Skateboard,null,10.7 pounds,null,https://images-na.ssl-images-amazon.com/images/I/51j3fPQTQkL.jpg|https://images-na.ssl-images-amazon.com/images/I/31hKM3cSoSL.jpg|https://images-na.ssl-images-amazon.com/images/I/51WlHdwghfL.jpg|https://images-na.ssl-images-amazon.com/images/I/51FsyLRBzwL.jpg|https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/transparent-pixel.jpg,https://www.amazon.com/DB-Longboards-CoreFlex-Fiberglass-Longboard/dp/B07KMVJJK7|https://www.amazon.com/DB-Longboards-CoreFlex-Fiberglass-Longboard/dp/B07KMN5KS7|https://www.amazon.com/DB-Longboards-CoreFlex-Fiberglass-Longboard/dp/B07KMXK857|https://www.amazon.com/DB-Longboards-CoreFlex-Fiberglass-Longboard/dp/B07KMW2VFR,null,https://www.amazon.com/DB-Longboards-CoreFlex-Fiberglass-Longboard/dp/B07KMVJJK7,null,null,null,null,null,null,Y,null,null
66d49bbed043f5be260fa9f7fbff5957,"Electronic Snap Circuits Mini Kits Classpack, FM Radio, Motion Detector, Music Box (Set of 5)",null,null,Toys & Games | Learning & Education | Science Kits & Toys,null,null,$99.95,null,55324,"Make sure this fits by entering your model number. | Snap circuits mini kits classpack provides basic electronic circuitry activities for students in grades 2-6 | Includes 5 separate mini building kits- an FM radio, a motion detector, music box, space battle sound effects, and a flying saucer | Each kit includes separate components and instructions to build | Each component represents one function in a circuit; components snap together to create working models of everyday electronic devices | Activity guide provides additional projects to teach students how circuitry works",Product Dimensions: 14.7 x 11.1 x 10.2 inches ; 4.06 pounds |Shipping Weight: 4 pounds (View shipping rates and policies)|Domestic Shipping: Item can be shipped within U.S.|International Shipping: This item can be shipped to select countries outside of the U.S. Learn More|ASIN: B008AK6DAS|Item model number: 55324| #3032 in Science Kits & Toys,"The snap circuits mini kits classpack provides basic electric circuitry information for students in grades 2-

Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


## Step 2: Bronze Layer

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.bronze;
CREATE SCHEMA IF NOT EXISTS workspace.silver;
CREATE SCHEMA IF NOT EXISTS workspace.gold;

### 2.1 Standardize column names to snake_case

In [0]:

#  step 1: Define a column-normalization function
import re
from pyspark.sql import functions as F

# Clean column names
def to_snake(s: str) -> str:
    s = s.strip().lower() # strip space before and after and convert to lowercase
    s = re.sub(r"[^\w]+", "_", s)   # replace spaces/symbols with _
    s = re.sub(r"_+", "_", s) # replace multiple _ with one
    return s.strip("_") # remove leading and trailing _

def snake_case_df(df):
    for c in df.columns:
        df = df.withColumnRenamed(c, to_snake(c))
    return df

# Step 2: Apply it to all threee raw tables
sales_raw_clean = snake_case_df(sales_raw)
products_raw_clean = snake_case_df(products_raw)
customers_raw_clean = snake_case_df(customers_raw)

display(sales_raw_clean.limit(5))
display(products_raw_clean.limit(5))
display(customers_raw_clean.limit(5))

user_id,product_id,interaction_type,time_stamp
1,4c69b61db1fc16e7013b43fc926e502d,purchase,10/10/2023 8:00
2,66d49bbed043f5be260fa9f7fbff5957,view,11/10/2023 8:00
3,2c55cae269aebf53838484b0d7dd931a,like,12/10/2023 8:00
4,18018b6bc416dab347b1b7db79994afa,view,13/10/2023 8:00
5,e04b990e95bf73bbe6a3fa09785d7cd0,like,14/10/2023 8:00


uniqe_id,product_name,brand_name,asin,category,upc_ean_code,list_price,selling_price,quantity,model_number,about_product,product_specification,technical_details,shipping_weight,product_dimensions,image,variants,sku,product_url,stock,product_details,dimensions,color,ingredients,direction_to_use,is_amazon_seller,size_quantity_variant,product_description
4c69b61db1fc16e7013b43fc926e502d,"DB Longboards CoreFlex Crossbow 41"" Bamboo Fiberglass Longboard Complete",null,null,"Sports & Outdoors | Outdoor Recreation | Skates, Skateboards & Scooters | Skateboarding | Standard Skateboards & Longboards | Longboards",null,null,$237.68,null,null,"Make sure this fits by entering your model number. | RESPONSIVE FLEX: The Crossbow features a bamboo core encased in triaxial fiberglass and HD plastic for a responsive flex pattern that’s second to none. Pumping & carving have never been so satisfying! Flex 2 is recommended for people 120 to 170 pounds. | COREFLEX TECH: CoreFlex construction is water resistant, impact resistant, scratch resistant and has a flex like you won’t believe. These boards combine fiberglass, epoxy, HD plastic and bamboo to create a perfect blend of performance and strength. | INSPIRED BY THE NORTHWEST: Our founding ideal is chasing adventure & riding the best boards possible, inspired by the hills, waves, beaches & mountains all around our headquarters in the Northwest | BEST IN THE WORLD: DB was founded out of sheer love of longboarding with a mission to create the best custom longboards in the world, to do it sustainably, & to treat customers & employees like family | BEYOND COMPARE: Try our skateboards & accessories if you've tried similar products by Sector 9, Landyachtz, Arbor, Loaded, Globe, Orangatang, Hawgs, Powell-Peralta, Blood Orange, Caliber or Gullwing",Shipping Weight: 10.7 pounds (View shipping rates and policies)|ASIN: B07KMVJJK7| #474 in Longboards Skateboard,null,10.7 pounds,null,https://images-na.ssl-images-amazon.com/images/I/51j3fPQTQkL.jpg|https://images-na.ssl-images-amazon.com/images/I/31hKM3cSoSL.jpg|https://images-na.ssl-images-amazon.com/images/I/51WlHdwghfL.jpg|https://images-na.ssl-images-amazon.com/images/I/51FsyLRBzwL.jpg|https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/transparent-pixel.jpg,https://www.amazon.com/DB-Longboards-CoreFlex-Fiberglass-Longboard/dp/B07KMVJJK7|https://www.amazon.com/DB-Longboards-CoreFlex-Fiberglass-Longboard/dp/B07KMN5KS7|https://www.amazon.com/DB-Longboards-CoreFlex-Fiberglass-Longboard/dp/B07KMXK857|https://www.amazon.com/DB-Longboards-CoreFlex-Fiberglass-Longboard/dp/B07KMW2VFR,null,https://www.amazon.com/DB-Longboards-CoreFlex-Fiberglass-Longboard/dp/B07KMVJJK7,null,null,null,null,null,null,Y,null,null
66d49bbed043f5be260fa9f7fbff5957,"Electronic Snap Circuits Mini Kits Classpack, FM Radio, Motion Detector, Music Box (Set of 5)",null,null,Toys & Games | Learning & Education | Science Kits & Toys,null,null,$99.95,null,55324,"Make sure this fits by entering your model number. | Snap circuits mini kits classpack provides basic electronic circuitry activities for students in grades 2-6 | Includes 5 separate mini building kits- an FM radio, a motion detector, music box, space battle sound effects, and a flying saucer | Each kit includes separate components and instructions to build | Each component represents one function in a circuit; components snap together to create working models of everyday electronic devices | Activity guide provides additional projects to teach students how circuitry works",Product Dimensions: 14.7 x 11.1 x 10.2 inches ; 4.06 pounds |Shipping Weight: 4 pounds (View shipping rates and policies)|Domestic Shipping: Item can be shipped within U.S.|International Shipping: This item can be shipped to select countries outside of the U.S. Learn More|ASIN: B008AK6DAS|Item model number: 55324| #3032 in Science Kits & Toys,"The snap circuits mini kits classpack provides basic electric circuitry information for students in grades 2-

customer_id,age,gender,item_purchased,category,purchase_amount_usd,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases
1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [0]:
# Validate that the columns are cleaned to snake_case
print(products_raw.columns)
print(products_raw_clean.columns)


['Uniqe Id', 'Product Name', 'Brand Name', 'Asin', 'Category', 'Upc Ean Code', 'List Price', 'Selling Price', 'Quantity', 'Model Number', 'About Product', 'Product Specification', 'Technical Details', 'Shipping Weight', 'Product Dimensions', 'Image', 'Variants', 'Sku', 'Product Url', 'Stock', 'Product Details', 'Dimensions', 'Color', 'Ingredients', 'Direction To Use', 'Is Amazon Seller', 'Size Quantity Variant', 'Product Description']
['uniqe_id', 'product_name', 'brand_name', 'asin', 'category', 'upc_ean_code', 'list_price', 'selling_price', 'quantity', 'model_number', 'about_product', 'product_specification', 'technical_details', 'shipping_weight', 'product_dimensions', 'image', 'variants', 'sku', 'product_url', 'stock', 'product_details', 'dimensions', 'color', 'ingredients', 'direction_to_use', 'is_amazon_seller', 'size_quantity_variant', 'product_description']


In [0]:
# Write Bronze Tables
(sales_raw_clean.write.format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("workspace.bronze.sales_interactions_raw"))

(products_raw_clean.write.format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("workspace.bronze.product_details_raw"))

(customers_raw_clean.write
  .format("delta")
  .mode("overwrite")
  .option("overwriteSchema", "true")
  .saveAsTable("workspace.bronze.customer_details_raw"))

## Step 3: Silver Layer

### 3.1 Sales Table

Step 1: Clean and normalize bronze_sales_interactions_raw table
1. drop empty column
2. cast user_id to int
3. parse timestamp
4. normalize interaction_type
5. deduplicate (same user, product, interaction, timestamp)

In [0]:
# Check current catalog and schema
spark.sql("SELECT current_catalog(), current_schema()").show(truncate=False) 


+-----------------+----------------+
|current_catalog()|current_schema()|
+-----------------+----------------+
|workspace        |default         |
+-----------------+----------------+



In [0]:
from pyspark.sql import functions as F

db = "workspace.bronze"

spark.sql(f"USE {db}")

sales_bronze = spark.table("workspace.bronze.sales_interactions_raw")

# Drop completely empty columns (like unnamed_4)
non_empty_cols = [c for c in sales_bronze.columns if sales_bronze.filter(F.col(c).isNotNull()).limit(1).count() > 0]
sales = sales_bronze.select(*non_empty_cols)

sales_silver = (sales
    .withColumn("user_id", F.col("user_id").cast("int")) # cast user_id to int
    .withColumn("event_ts", F.to_timestamp("time_stamp", "d/M/yyyy H:mm"))  # like 10/10/2023 8:00
    .drop("time_stamp")
    .filter(F.col("user_id").isNotNull())
    .filter(F.col("product_id").isNotNull())
    .filter(F.col("event_ts").isNotNull())
)

# Deduplicate exact duplicates
sales_silver = sales_silver.dropDuplicates(["user_id", "product_id", "interaction_type", "event_ts"])

# Add event_date for partition-friendly analytics
sales_silver = sales_silver.withColumn("event_date", F.to_date("event_ts")) # convert timestamp to date

# Confirm schema standerization
sales_silver.printSchema()
# (nullable = true) indicates that the column may contain null values at the schema level, even if nulls have been filtered out during transformations.

# write data to delta format
(sales_silver.write.format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("workspace.silver.sales_interactions"))


root
 |-- user_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- interaction_type: string (nullable = true)
 |-- event_ts: timestamp (nullable = true)
 |-- event_date: date (nullable = true)



In [0]:
%sql
SELECT * FROM workspace.silver.sales_interactions LIMIT 10;


user_id,product_id,interaction_type,event_ts,event_date
681,ae52ba51dba91bcbe1b2a4815903c40a,like,2023-01-09T10:00:00.000Z,2023-01-09
2413,12658e8bae58486bfa2a2885380c07cb,view,2023-12-07T08:00:00.000Z,2023-12-07
2806,88ea1baa294283526e838357de9dd110,view,2023-09-15T10:00:00.000Z,2023-09-15
2632,165e609717373146f50529986ded897d,like,2023-03-25T10:00:00.000Z,2023-03-25
2873,87f78a1ba957dde09300521b7320374e,like,2023-12-02T08:00:00.000Z,2023-12-02
2040,01bacf123137341fc0b1e86aec29c3e7,like,2023-11-22T08:00:00.000Z,2023-11-22
2424,75d64da1aa9c1f6fcd9d0a2bf20a71dd,like,2023-12-18T08:00:00.000Z,2023-12-18
2430,95e0cd18ad6c6379fd03351480e806b3,purchase,2023-12-24T08:00:00.000Z,2023-12-24
1293,060c4fbc66821a1268e65a65dd877f36,like,2023-10-30T08:00:00.000Z,2023-10-30
550,0b875d54bbfc8902a6f053db9b395052,view,2023-12-20T08:00:00.000Z,2023-12-20


In [0]:
# How many users have interacted with the website?
spark.table("workspace.silver.sales_interactions").select(
    F.count("*").alias("rows"),
    F.countDistinct("user_id").alias("distinct_users"),
    # isNull() returns true = 1, false = 0
    F.sum(F.col("user_id").isNull().cast("int")).alias("null_user_id")
).show()


+----+--------------+------------+
|rows|distinct_users|null_user_id|
+----+--------------+------------+
|2999|          2999|           0|
+----+--------------+------------+



In [0]:
%sql
-- Sanity check
-- Show table level metadata
DESCRIBE DETAIL workspace.silver.sales_interactions;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,9e206abd-2aaf-45e2-ae2e-93f404eb8ce7,workspace.silver.sales_interactions,null,,2026-01-24T22:56:19.330Z,2026-01-24T22:56:21.000Z,List(),List(),1,71326,"Map(delta.parquet.compression.codec -> zstd, delta.enableDeletionVectors -> true)",3,7,"List(appendOnly, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false


In [0]:
%sql
DESCRIBE HISTORY workspace.silver.sales_interactions;

-- Select from a specific version
-- SELECT * 
-- FROM workspace.silver.sales_interactions VERSION AS OF 5;

-- Restore to a previous version
-- RESTORE TABLE workspace.silver.sales_interactions TO VERSION AS OF 5;



version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2026-01-24T22:56:21.000Z,76408141857361,milliepwa@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(399061186738051),0124-201450-ju2kpwa6-v2n,null,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 0, numRemovedBytes -> 0, numDeletionVectorsRemoved -> 0, numOutputRows -> 2999, numOutputBytes -> 71326)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13


In [0]:
%sql
SELECT 
  (SELECT COUNT(*) FROM workspace.bronze.sales_interactions_raw) AS bronze_rows,
  (SELECT COUNT(*) FROM workspace.silver.sales_interactions) AS silver_rows;

bronze_rows,silver_rows
3294,2999


In [0]:
%sql
DROP TABLE workspace.bronze.silver_sales_interactions;


### 3.2 Products Table

Step 1: Clean and normalize bronze_product_details_raw
1. standardize columns
2. cast prices to decimals (strip currency symbols if any)
3. deduplicate by product id (uniqe_id)
4. keep a clean product dimension table

In [0]:
# from pyspark.sql import functions as F
from pyspark.sql.window import Window

# spark.sql("USE workspace.default")

products = spark.table("workspace.bronze.product_details_raw")

# Ensure product_id column exists in the right name
products = products.withColumnRenamed("uniqe_id", "product_id")

# Drop completely empty columns
non_empty_cols = [c for c in products.columns if products.filter(F.col(c).isNotNull()).limit(1).count() > 0]
products = products.select(*non_empty_cols)

print("Columns after selection:", products.columns)

def clean_price(colname):
    raw = F.col(colname).cast("string")
    s = F.regexp_replace(raw, r"[^0-9.]", "") # Remove all non-digits and dots
    s = F.when(s.rlike(r"^\.*$"), F.lit(None)).otherwise(s) # Remove leading dots

    parts = F.split(s, r"\.")
    s = F.when(
        s.isNull(), F.lit(None)
    ).when(
        F.size(parts) >= 2, F.concat_ws(".", parts.getItem(0), parts.getItem(1))
    ).otherwise(s)

    s = F.when((s.isNull()) | (F.length(s) == 0) | (s.rlike(r"^\.*$")), F.lit(None)).otherwise(s)
    return s.cast("decimal(12,2)")

products_silver = products

if "selling_price" in products.columns:
    products_silver = products_silver.withColumn("selling_price", clean_price("selling_price"))

# Deduplicate: keep best record per product_id (prefer higher selling_price)
w = Window.partitionBy("product_id").orderBy(F.col("selling_price").desc_nulls_last())
products_silver = (products_silver
    .withColumn("rn", F.row_number().over(w))
    .filter(F.col("rn") == 1)
    .drop("rn")
)

products_silver.printSchema()

# write data to delta format
(products_silver.write.format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("workspace.silver.products"))


Columns after selection: ['product_id', 'product_name', 'category', 'upc_ean_code', 'selling_price', 'model_number', 'about_product', 'product_specification', 'technical_details', 'shipping_weight', 'product_dimensions', 'image', 'variants', 'product_url', 'is_amazon_seller']
root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- upc_ean_code: string (nullable = true)
 |-- selling_price: decimal(12,2) (nullable = true)
 |-- model_number: string (nullable = true)
 |-- about_product: string (nullable = true)
 |-- product_specification: string (nullable = true)
 |-- technical_details: string (nullable = true)
 |-- shipping_weight: string (nullable = true)
 |-- product_dimensions: string (nullable = true)
 |-- image: string (nullable = true)
 |-- variants: string (nullable = true)
 |-- product_url: string (nullable = true)
 |-- is_amazon_seller: string (nullable = true)



In [0]:
%sql
SELECT * FROM workspace.silver.products LIMIT 10;


product_id,product_name,category,upc_ean_code,selling_price,model_number,about_product,product_specification,technical_details,shipping_weight,product_dimensions,image,variants,product_url,is_amazon_seller
0000fe97fd6c7705b08b7f4c7c5312ce,Banpresto 39652 Dragon Ball Super Blood of Saiyans Special VI S.God Goku Figure,null,null,17.90,39652,Make sure this fits by entering your model number. | Official product by little buddy/ Banpresto | Cute and collectible | Makes a great gift | Limited availability,ProductDimensions:4x4x6.7inches|ItemWeight:8.8ounces|ShippingWeight:12.6ounces(Viewshippingratesandpolicies)|ASIN:B07S5QJCLD|Itemmodelnumber:39652|Manufacturerrecommendedage:13-15years,"Go to your orders and start the return Select the ship method Ship it! | Go to your orders and start the return Select the ship method Ship it! | ""Dragon ball Z"" Follows the adventures of Goku who, along with the Z warriors, defends the earth against evil. The action adventures are entertaining and reinforce the concept of good versus evil. ""Dragon Ball Z"" Teaches valuable character virtues such as teamwork, loyalty, and trustworthiness. | 12.6 ounces (View shipping rates and policies)",12.6 ounces,null,https://images-na.ssl-images-amazon.com/images/I/418XikMmXML.jpg|https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/transparent-pixel.jpg,null,https://www.amazon.com/Banpresto-39652-Dragon-Saiyans-Special/dp/B07S5QJCLD,Y
00037729f1cb566e149f0beff2d9ffd3,Pokeno 2,Toys & Games | Games & Accessories | Card Games,null,15.01,Pokeno(Return),"Make sure this fits by entering your model number. | American puzzles, 200 Poke no chips | Playing cards are not included | 12 Poke no too Bicycle Jumbo index game boards | Each board size is 9 1/2"" X 7"" With pictures of 25 cards | Each card picture is 1 3/4"" High x 1 1/4""wide","ProductDimensions:10.1x9.6x1inches|ItemWeight:1pounds|ShippingWeight:1.3pounds(Viewshippingratesandpolicies)|DomesticShipping:Currently,itemcanbeshippedonlywithintheU.S.andtoAPO/FPOaddresses.ForAPO/FPOshipments,pleasecheckwiththemanufacturerregardingwarrantyandsupportissues.|InternationalShipping:ThisitemcanbeshippedtoselectcountriesoutsideoftheU.S.LearnMore|ASIN:B0009XBXE0|Itemmodelnumber:Pokeno(Return)|Manufacturerrecommendedage:24months-7years","Go to your orders and start the return Select the ship method Ship it! | Go to your orders and start the return Select the ship method Ship it! | show up to 2 reviews by default If you already have the original Poke no and would like to play with more than 12 people, then order the Poke no too. Jumbo 7"" X 9-1/2"" Game boards make play easier and like bingo, players cover the cards on their Boards to finish lines, four corners or cover all. Includes 12 Poke no too boards with 200 Poke no cover chips. The board design is different than original Poke no to allow more people to play. | 1.3 pounds (View shipping rates and policies)",1.3 pounds,null,https://images-na.ssl-images-amazon.com/images/I/51p2WOfr5PL.jpg|https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/transparent-pixel.jpg,null,https://www.amazon.com/Bicycle-Pokeno-Return-2/dp/B0009XBXE0,Y
0003a7ccfb779e7ac1664fb13145aff2,Medicom JUL188832 Peanuts: Surfer Snoopy Ultra Detail Figure,null,null,9.94,-,Make sure this fits by entering your model number. | A Medicom import | Celebrate the unmatched creativity of Charles schulz | Blister card packaging | Collect them all,ProductDimensions:4x1.5x1.5inches|ItemWeight:3.52ounces|ShippingWeight:4.6ounces|ASIN:B07GNCPFJ1|Itemmodelnumber:-|Manufacturerrecommendedage:15yearsandup,show up to 2 reviews by default From Medicom toy. Medicom salutes everyone's favorite Beagle and his friends with these great Peanuts ultra detail figures! | 4.6 ounces,4.6 ounces,null,https://images-na.ssl-images-amazon.com/images/I/31LF-0OcNdL.jpg|https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/transparent-pixel.jpg,null,https://www.amazon.com/Medicom-JUL188832-Peanuts

In [0]:
spark.table("workspace.silver.products").select(
    F.count("*").alias("rows"),
    F.countDistinct("product_id").alias("distinct_products"),
    F.sum(F.col("product_id").isNull().cast("int")).alias("null_product_id")
).show()


+-----+-----------------+---------------+
| rows|distinct_products|null_product_id|
+-----+-----------------+---------------+
|10002|            10002|              0|
+-----+-----------------+---------------+



In [0]:
%sql
SELECT 
  (SELECT COUNT(*) FROM workspace.bronze.product_details_raw) AS bronze_rows,
  (SELECT COUNT(*) FROM workspace.silver.products) AS silver_rows;


bronze_rows,silver_rows
10002,10002


### 3.3 Customers **Table**

In [0]:
%sql
--Sanity check
USE default;
DESCRIBE workspace.bronze.customer_details_raw;

col_name,data_type,comment
customer_id,bigint,null
age,bigint,null
gender,string,null
item_purchased,string,null
category,string,null
purchase_amount_usd,bigint,null
location,string,null
size,string,null
color,string,null
season,string,null


In [0]:
customers = spark.table("workspace.bronze.customer_details_raw")

# 1) Canonicalize the user id column
# Detect a usable ID column
id_candidates = ["user_id", "customer_id", "customerid", "customer_id_", "id", "customer"]
id_col = next((c for c in id_candidates if c in customers.columns), None)

if id_col is None:
    raise Exception(f"No customer id column found. Columns are: {customers.columns}")

# Create canonical user_id as INT
customers = customers.withColumn("user_id", F.col(id_col).cast("int"))

# Drop the old id column if it's different from user_id (keeps schema clean)
if id_col != "user_id" and id_col in customers.columns:
    customers = customers.drop(id_col)

# 2) Normalize common text fields
# Standardize gender if present
if "gender" in customers.columns:
    customers = customers.withColumn(
        "gender",
        F.when(F.col("gender").isNull(), None) # if none, keep it
         .otherwise(F.initcap(F.lower(F.trim(F.col("gender"))))) # to Female/Male
    )

# Standardize location if present
if "location" in customers.columns:
    customers = customers.withColumn("location", F.trim(F.col("location")))

# Cast age if present
if "age" in customers.columns:
    customers = customers.withColumn("age", F.col("age").cast("int"))

# Convert common yes/no flags to boolean if they exist
yes_no_cols = ["subscription_status", "discount_applied", "promo_code_used"]
for c in yes_no_cols:
    if c in customers.columns:
        customers = customers.withColumn(
            c,
            F.when(F.lower(F.trim(F.col(c))) == "yes", F.lit(True))
             .when(F.lower(F.trim(F.col(c))) == "no", F.lit(False))
             .otherwise(None)
        )

# 3) Filter bad ids + deduplicate
customers_silver = (customers
    .filter(F.col("user_id").isNotNull())
    .dropDuplicates(["user_id"])
)

# 3.5) Reorder columns: put user_id first
cols = customers_silver.columns
customers_silver = customers_silver.select(
    ["user_id"] + [c for c in cols if c != "user_id"]
)

# 4) Write Silver
(customers_silver.write.format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("workspace.silver.customers"))


In [0]:
%sql
SELECT * FROM workspace.silver.customers LIMIT 10;


user_id,age,gender,item_purchased,category,purchase_amount_usd,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases
1021,24,Male,Scarf,Accessories,95,Delaware,M,Black,Fall,4.5,true,Express,true,true,41,Debit Card,Annually
2737,44,Female,Hoodie,Clothing,25,Montana,M,Pink,Spring,4.8,false,Store Pickup,false,false,45,Credit Card,Annually
2936,58,Female,Sneakers,Footwear,27,North Carolina,XL,Gray,Spring,4.6,false,Standard,false,false,11,Venmo,Weekly
3139,47,Female,Blouse,Clothing,95,Kentucky,L,Gray,Fall,4.6,false,Next Day Air,false,false,40,Cash,Every 3 Months
3683,42,Female,Shirt,Clothing,60,Wyoming,M,Maroon,Fall,4.6,false,Store Pickup,false,false,46,Venmo,Bi-Weekly
94,37,Male,Sweater,Clothing,62,Florida,M,Purple,Fall,4.5,true,Standard,true,true,46,Credit Card,Weekly
1721,25,Male,Shorts,Clothing,84,Massachusetts,S,Beige,Winter,2.8,false,Express,false,false,40,Debit Card,Bi-Weekly
1007,21,Male,Pants,Clothing,83,Michigan,S,Lavender,Summer,4.5,true,Next Day Air,true,true,50,Venmo,Quarterly
1257,45,Male,Coat,Outerwear,48,West Virginia,S,Olive,Winter,3.6,false,Express,true,true,4,Cash,Fortnightly
1329,37,Male,Sweater,Clothing,97,Rhode Island,M,Orange,Winter,3.7,false,Standard,true,true,40,Bank Transfer,Weekly


In [0]:
%sql
SELECT 
  (SELECT COUNT(*) FROM workspace.bronze.customer_details_raw) AS bronze_rows,
  (SELECT COUNT(*) FROM workspace.silver.customers) AS silver_rows;


bronze_rows,silver_rows
3900,3900


In [0]:
spark.table("workspace.silver.customers").select(
    F.count("*").alias("rows"),
    F.countDistinct("user_id").alias("distinct_users"),
    F.sum(F.col("user_id").isNull().cast("int")).alias("null_user_id")
).show()


+----+--------------+------------+
|rows|distinct_users|null_user_id|
+----+--------------+------------+
|3900|          3900|           0|
+----+--------------+------------+



## Step 4: Gold Layer


In [0]:
%sql
SHOW TABLES IN workspace.silver;

database,tableName,isTemporary
silver,customers,false
silver,products,false
silver,sales_interactions,false


In [0]:
spark.table("workspace.silver.sales_interactions").printSchema()
spark.table("workspace.silver.products").printSchema()
spark.table("workspace.silver.customers").printSchema()


root
 |-- user_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- interaction_type: string (nullable = true)
 |-- event_ts: timestamp (nullable = true)
 |-- event_date: date (nullable = true)

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- upc_ean_code: string (nullable = true)
 |-- selling_price: decimal(12,2) (nullable = true)
 |-- model_number: string (nullable = true)
 |-- about_product: string (nullable = true)
 |-- product_specification: string (nullable = true)
 |-- technical_details: string (nullable = true)
 |-- shipping_weight: string (nullable = true)
 |-- product_dimensions: string (nullable = true)
 |-- image: string (nullable = true)
 |-- variants: string (nullable = true)
 |-- product_url: string (nullable = true)
 |-- is_amazon_seller: string (nullable = true)

root
 |-- user_id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: str

### 4.1 Build a curated gold fact table: fact_interactions


Base: silver.sales_interactions (events)
- Join products on product_id
- Join customers on user_id
- Derived fields: view/like/purchase flags + derived revenue

`Assumption`: 1 purchase event = 1 unit, revenue = selling_price

In [0]:
display(spark.table("workspace.silver.sales_interactions").limit(5))
display(spark.table("workspace.silver.products").limit(5))
display(spark.table("workspace.silver.customers").limit(5))

user_id,product_id,interaction_type,event_ts,event_date
681,ae52ba51dba91bcbe1b2a4815903c40a,like,2023-01-09T10:00:00.000Z,2023-01-09
2413,12658e8bae58486bfa2a2885380c07cb,view,2023-12-07T08:00:00.000Z,2023-12-07
2806,88ea1baa294283526e838357de9dd110,view,2023-09-15T10:00:00.000Z,2023-09-15
2632,165e609717373146f50529986ded897d,like,2023-03-25T10:00:00.000Z,2023-03-25
2873,87f78a1ba957dde09300521b7320374e,like,2023-12-02T08:00:00.000Z,2023-12-02


product_id,product_name,category,upc_ean_code,selling_price,model_number,about_product,product_specification,technical_details,shipping_weight,product_dimensions,image,variants,product_url,is_amazon_seller
0000fe97fd6c7705b08b7f4c7c5312ce,Banpresto 39652 Dragon Ball Super Blood of Saiyans Special VI S.God Goku Figure,null,null,17.90,39652,Make sure this fits by entering your model number. | Official product by little buddy/ Banpresto | Cute and collectible | Makes a great gift | Limited availability,ProductDimensions:4x4x6.7inches|ItemWeight:8.8ounces|ShippingWeight:12.6ounces(Viewshippingratesandpolicies)|ASIN:B07S5QJCLD|Itemmodelnumber:39652|Manufacturerrecommendedage:13-15years,"Go to your orders and start the return Select the ship method Ship it! | Go to your orders and start the return Select the ship method Ship it! | ""Dragon ball Z"" Follows the adventures of Goku who, along with the Z warriors, defends the earth against evil. The action adventures are entertaining and reinforce the concept of good versus evil. ""Dragon Ball Z"" Teaches valuable character virtues such as teamwork, loyalty, and trustworthiness. | 12.6 ounces (View shipping rates and policies)",12.6 ounces,null,https://images-na.ssl-images-amazon.com/images/I/418XikMmXML.jpg|https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/transparent-pixel.jpg,null,https://www.amazon.com/Banpresto-39652-Dragon-Saiyans-Special/dp/B07S5QJCLD,Y
00037729f1cb566e149f0beff2d9ffd3,Pokeno 2,Toys & Games | Games & Accessories | Card Games,null,15.01,Pokeno(Return),"Make sure this fits by entering your model number. | American puzzles, 200 Poke no chips | Playing cards are not included | 12 Poke no too Bicycle Jumbo index game boards | Each board size is 9 1/2"" X 7"" With pictures of 25 cards | Each card picture is 1 3/4"" High x 1 1/4""wide","ProductDimensions:10.1x9.6x1inches|ItemWeight:1pounds|ShippingWeight:1.3pounds(Viewshippingratesandpolicies)|DomesticShipping:Currently,itemcanbeshippedonlywithintheU.S.andtoAPO/FPOaddresses.ForAPO/FPOshipments,pleasecheckwiththemanufacturerregardingwarrantyandsupportissues.|InternationalShipping:ThisitemcanbeshippedtoselectcountriesoutsideoftheU.S.LearnMore|ASIN:B0009XBXE0|Itemmodelnumber:Pokeno(Return)|Manufacturerrecommendedage:24months-7years","Go to your orders and start the return Select the ship method Ship it! | Go to your orders and start the return Select the ship method Ship it! | show up to 2 reviews by default If you already have the original Poke no and would like to play with more than 12 people, then order the Poke no too. Jumbo 7"" X 9-1/2"" Game boards make play easier and like bingo, players cover the cards on their Boards to finish lines, four corners or cover all. Includes 12 Poke no too boards with 200 Poke no cover chips. The board design is different than original Poke no to allow more people to play. | 1.3 pounds (View shipping rates and policies)",1.3 pounds,null,https://images-na.ssl-images-amazon.com/images/I/51p2WOfr5PL.jpg|https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/transparent-pixel.jpg,null,https://www.amazon.com/Bicycle-Pokeno-Return-2/dp/B0009XBXE0,Y
0003a7ccfb779e7ac1664fb13145aff2,Medicom JUL188832 Peanuts: Surfer Snoopy Ultra Detail Figure,null,null,9.94,-,Make sure this fits by entering your model number. | A Medicom import | Celebrate the unmatched creativity of Charles schulz | Blister card packaging | Collect them all,ProductDimensions:4x1.5x1.5inches|ItemWeight:3.52ounces|ShippingWeight:4.6ounces|ASIN:B07GNCPFJ1|Itemmodelnumber:-|Manufacturerrecommendedage:15yearsandup,show up to 2 reviews by default From Medicom toy. Medicom salutes everyone's favorite Beagle and his friends with these great Peanuts ultra detail figures! | 4.6 ounces,4.6 ounces,null,https://images-na.ssl-images-amazon.com/images/I/31LF-0OcNdL.jpg|https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/transparent-pixel.jpg,null,https://www.amazon.com/Medicom-JUL188832-Peanuts

user_id,age,gender,item_purchased,category,purchase_amount_usd,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases
1021,24,Male,Scarf,Accessories,95,Delaware,M,Black,Fall,4.5,true,Express,true,true,41,Debit Card,Annually
2737,44,Female,Hoodie,Clothing,25,Montana,M,Pink,Spring,4.8,false,Store Pickup,false,false,45,Credit Card,Annually
2936,58,Female,Sneakers,Footwear,27,North Carolina,XL,Gray,Spring,4.6,false,Standard,false,false,11,Venmo,Weekly
3139,47,Female,Blouse,Clothing,95,Kentucky,L,Gray,Fall,4.6,false,Next Day Air,false,false,40,Cash,Every 3 Months
3683,42,Female,Shirt,Clothing,60,Wyoming,M,Maroon,Fall,4.6,false,Store Pickup,false,false,46,Venmo,Bi-Weekly


In [0]:
%sql
SELECT user_id, COUNT(*) cnt
FROM workspace.silver.customers
GROUP BY user_id
HAVING cnt > 1;


user_id,cnt


In [0]:
%sql
-- Find out what are the interaction_types and their counts in the sales_interactions table
SELECT interaction_type, COUNT(*) AS cnt
FROM workspace.silver.sales_interactions
GROUP BY interaction_type
ORDER BY cnt DESC;


interaction_type,cnt
like,1145
view,871
purchase,855
null,128


In [0]:
%sql
--  How many users are in both sales_interactions and customers table
SELECT
  (SELECT COUNT(DISTINCT user_id) FROM workspace.silver.sales_interactions) AS users_in_interactions,
  (SELECT COUNT(DISTINCT user_id) FROM workspace.silver.customers) AS users_in_customers,
  (SELECT COUNT(DISTINCT si.user_id)
     FROM workspace.silver.sales_interactions si
     INNER JOIN workspace.silver.customers c
       ON si.user_id = c.user_id
  ) AS users_in_both;


users_in_interactions,users_in_customers,users_in_both
2999,3900,2999


In [0]:
%sql
-- When purchase_cnt = 0, there are still customers.item_purchased and customers.payment_method
-- This shows that the customers table have a different definition of purchase from the sales_interaction table
-- that we are not sure about
WITH p AS (
  SELECT user_id, COUNT(*) AS purchase_cnt
  FROM workspace.silver.sales_interactions
  WHERE interaction_type = 'purchase'
  GROUP BY user_id
)
SELECT
  c.user_id,
  p.purchase_cnt,
  c.item_purchased,
  c.purchase_amount_usd,
  c.discount_applied,
  c.payment_method,
  c.shipping_type
FROM workspace.silver.customers c
LEFT JOIN p ON c.user_id = p.user_id
LIMIT 50;


user_id,purchase_cnt,item_purchased,purchase_amount_usd,discount_applied,payment_method,shipping_type
1021,null,Scarf,95,true,Debit Card,Express
2737,1,Hoodie,25,false,Credit Card,Store Pickup
2936,null,Sneakers,27,false,Venmo,Standard
3139,null,Blouse,95,false,Cash,Next Day Air
3683,null,Shirt,60,false,Venmo,Store Pickup
94,1,Sweater,62,true,Credit Card,Standard
1721,1,Shorts,84,false,Debit Card,Express
1007,null,Pants,83,true,Venmo,Next Day Air
1257,1,Coat,48,true,Cash,Express
1329,null,Sweater,97,true,Bank Transfer,Standard


In [0]:
# spark.sql("USE workspace.default")
from pyspark.sql import functions as F

sales = spark.table("workspace.silver.sales_interactions")
products = spark.table("workspace.silver.products")
customers = spark.table("workspace.silver.customers")

# Keep only the columns we need from dimensions (avoid accidental duplicates)
prod_dim = products.select(
    "product_id",
    "product_name",
    "category",
    "selling_price",
    "is_amazon_seller"
)

# Customers table:
# keep only the columns that are not tied to a specific purchase, such as excluding item_purchased, purchase_amount_usd etc.
cust_cols = [c for c in ["user_id", "age", "gender", "location"] if c in customers.columns]
# cust_cols = [c for c in customers.columns]
cust_dim = customers.select(*cust_cols)

gold_fact = (sales
    .join(prod_dim, on="product_id", how="left") # left join这里对吗？
    .join(cust_dim, on="user_id", how="left")
    .withColumn("is_view", (F.col("interaction_type")=="view").cast("int"))
    .withColumn("is_like", (F.col("interaction_type")=="like").cast("int"))
    .withColumn("is_purchase", (F.col("interaction_type")=="purchase").cast("int"))
    .withColumn(
        "derived_revenue",
        F.when(F.col("interaction_type")=="purchase", F.col("selling_price")).otherwise(F.lit(0.0))
    )
)

(gold_fact.write.format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("workspace.gold.fact_interactions"))


In [0]:
%sql
SELECT * FROM workspace.gold.fact_interactions LIMIT 10;


user_id,product_id,interaction_type,event_ts,event_date,product_name,category,selling_price,is_amazon_seller,age,gender,location,is_view,is_like,is_purchase,derived_revenue
681,ae52ba51dba91bcbe1b2a4815903c40a,like,2023-01-09T10:00:00.000Z,2023-01-09,"Nino Percussion NINO-NS355 14-Inch Cymbal with Holding Straps, Nickel Silver",Toys & Games | Learning & Education | Musical Instruments | Drums & Percussion,52.69,Y,58,Male,Tennessee,0,1,0,0.0
2413,12658e8bae58486bfa2a2885380c07cb,view,2023-12-07T08:00:00.000Z,2023-12-07,Disney Mickey Popcorn Coral Fleece Blanket,Home & Kitchen | Bedding | Kids' Bedding | Blankets & Throws,20.02,Y,53,Male,Oregon,1,0,0,0.0
2806,88ea1baa294283526e838357de9dd110,view,2023-09-15T10:00:00.000Z,2023-09-15,Marvel Spider-Man Titan Hero Series Villains Carnage Figure,null,15.08,Y,18,Female,Nevada,1,0,0,0.0
2632,165e609717373146f50529986ded897d,like,2023-03-25T10:00:00.000Z,2023-03-25,"Funko 289 Animation: Pop! Rocky & Bullwinkle Collectors Magician Bullwinkle, Flying Rocky, Bullwinkle Fearlessleader, Set",Toys & Games | Toy Figures & Playsets | Action Figures,6.89,Y,53,Male,Connecticut,0,1,0,0.0
2873,87f78a1ba957dde09300521b7320374e,like,2023-12-02T08:00:00.000Z,2023-12-02,"Redcat Racing 02075 Steering Bell Crank, 1Piece",Hobbies | Remote & App Controlled Vehicles & Parts | Remote & App Controlled Vehicle Parts | Power Plant & Driveline Systems | Suspension Systems & Parts,6.79,Y,63,Female,Connecticut,0,1,0,0.0
2040,01bacf123137341fc0b1e86aec29c3e7,like,2023-11-22T08:00:00.000Z,2023-11-22,Fisher-Price Laugh & Learn Sweet Manners Tea Set,null,19.88,Y,35,Male,Minnesota,0,1,0,0.0
2424,75d64da1aa9c1f6fcd9d0a2bf20a71dd,like,2023-12-18T08:00:00.000Z,2023-12-18,"School Smart Blunt Tip Student Scissors, 6-1/4 Inches, Pack of 12",Office Products | Office & School Supplies | Education & Crafts | Students Scissors,25.32,Y,49,Male,West Virginia,0,1,0,0.0
2430,95e0cd18ad6c6379fd03351480e806b3,purchase,2023-12-24T08:00:00.000Z,2023-12-24,RepliCade Insert Coin Keychain,Toys & Games | Hobbies | Coin Collecting,14.99,Y,57,Male,Virginia,0,0,1,14.99
1293,060c4fbc66821a1268e65a65dd877f36,like,2023-10-30T08:00:00.000Z,2023-10-30,"Modular Robotics Cubelets Robot Blocks - Code & Construct Educator Pack - STEM Education & Coding Robot, Free Lesson Plans, Ages 4+",Toys & Games | Learning & Education | Science Kits & Toys,1734.00,Y,26,Male,Kentucky,0,1,0,0.0
550,0b875d54bbfc8902a6f053db9b395052,view,2023-12-20T08:00:00.000Z,2023-12-20,Rhode Island Novelty Monkey Bean Filled Plush Stuffed Animal,Home & Kitchen | Furniture | Kids' Furniture,9.99,Y,31,Male,Arizona,1,0,0,0.0


### 4.2 Build a gold products table: product_kpis

- views, likes, purchases
- revenue (derived)
- active_days

In [0]:
from pyspark.sql import functions as F

fact = spark.table("workspace.gold.fact_interactions")

gold_product_kpis = (fact.groupBy("product_id","product_name","category") # this results in each line in the export table representing a product
    .agg(
        F.sum("is_view").alias("views"), # count of all views of this product
        F.sum("is_like").alias("likes"), # count of all likes of this product
        F.sum("is_purchase").alias("purchases"), # count of all purchases of this product 
        F.sum("derived_revenue").alias("revenue"), # sum of all revenues from this product
        F.countDistinct("event_date").alias("active_days") # how many days have this product bee seen in interactions
    )
    .withColumn(
        "purchase_view_rate",
        F.when(F.col("views") > 0, F.col("purchases") / F.col("views")).otherwise(F.lit(None)) # set it to null if no views
    )
)

(gold_product_kpis.write.format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("workspace.gold.product_kpis"))


In [0]:
%sql
-- SELECT product_id, product_name, category, purchases, revenue
SELECT * 
FROM workspace.gold.product_kpis
ORDER BY revenue DESC
LIMIT 20;


product_id,product_name,category,views,likes,purchases,revenue,active_days,purchase_view_rate
0b9df9bc7f554b556776ce0cf62debfa,Turtle Beach XO Three Gaming Headset for Xbox One,Toys & Games | Dress Up & Pretend Play | Pretend Play,0,0,1,539859.95,1,null
1415fed85fb0f78111c2572fdf57dcd2,"Gorilla Playsets 01-3036-G Malibu Playhouse Wooden Playhouse for Kids, Brown & Green",Toys & Games | Sports & Outdoor Play | Play Sets & Playground Equipment | Play & Swing Sets,0,0,1,1265.0,1,null
198651a0f2df856435f4e7401d2329ca,DJI Smart Controller,Toys & Games | Hobbies | Remote & App Controlled Vehicles & Parts | Remote & App Controlled Vehicles | Quadcopters & Multirotors,0,0,1,718.98,1,null
034bc638e2ccd85bd37193d028202ecf,"Losi 1/10 2.2 Rock Rey 4WD RC Rock Racer Brushless RTR with AVC and 2800Kv Brushless Power System (Battery and Charger Not Included): LOS03009T1, Yellow",Toys & Games | Hobbies | Remote & App Controlled Vehicles & Parts | Remote & App Controlled Vehicles | Trucks,0,0,1,399.99,1,null
ef2b5e145ff5825a1f8bac4051a2a1a9,"Bugatti Chiron 12V Ride On Car. Premium Convertible Bugatti Chiron Style Ride on Car Kids Music,Lights, Doors, RC","Toys & Games | Tricycles, Scooters & Wagons | Ride-On Toys & Accessories",0,0,1,389.99,1,null
ae1147825ea16124bf61f5428c4ad964,Banzai Battle Blast Adventure Inflatable Water Park,Toys & Games | Sports & Outdoor Play | Pools & Water Toys | Pool Toys | Inflatable Water Slides,0,0,1,349.99,1,null
289cf4e51ba4f56bd0e6bb10ac068a0c,"LEGO Technic Bugatti Chiron 42083 Race Car Building Kit and Engineering Toy, Adult Collectible Sports Car with Scale Model Engine (3599 Pieces)",Toys & Games | Building Toys,0,0,1,347.78,1,null
0811c593514ef345df62b4bbd241d803,Connelly Party Cove Island Water Mat,Toys & Games | Sports & Outdoor Play | Pools & Water Toys,0,0,1,250.0,1,null
22bccbee6c18e566f04592da316e3c34,Rayne Longboards Demonseed Longboard Complete,"Sports & Outdoors | Outdoor Recreation | Skates, Skateboards & Scooters | Skateboarding | Standard Skateboards & Longboards | Longboards",0,0,1,249.95,1,null
89d180e819d5a779ddec1f8737488821,"Rollerblade Twister Edge Men's Adult Fitness Inline Skate, Black and Red, High Performance Inline Skates","Sports & Outdoors | Outdoor Recreation | Skates, Skateboards & Scooters | Inline & Roller Skating | Inline Skates",0,0,1,249.0,1,null


In [0]:
%sql
-- Find out how many interaction types are for each product
SELECT
  product_id,
  COUNT(DISTINCT interaction_type) AS interaction_types
FROM workspace.gold.fact_interactions
GROUP BY product_id
ORDER BY interaction_types DESC;

product_id,interaction_types
9c5d3643fa3670072302b9467acb3aa8,1
ceb83bfa91665801a1b91aef401fd799,1
7216fbdb76fd0d1cc445e6c421ee83a2,1
5caaf089f37595fdc5333946fcd0c56e,1
3d08b2296ba1c058d28b95785f1691c8,1
9dde85d0cfc1bfadc2421426113721af,1
3cee92e17483f36f7dfff6cb8f78cef2,1
e9d42f92d328569942f55ad2c9fb6f0e,1
02d5d3748dc98cf913b93dc8b5c05c8b,1
869ea8e7992197d5a0b80cdbd723227d,1


In [0]:
%sql
-- result show that only 1 interaction type for each product
SELECT COUNT(*) AS products_with_both
FROM (
  SELECT
    product_id,
    SUM(is_view) AS views,
    SUM(is_purchase) AS purchases
  FROM workspace.gold.fact_interactions
  GROUP BY product_id
) t
WHERE views > 0 AND purchases > 0;


products_with_both
0


### 4.3 Build gold daily_trends:

- total events
  - views/likes/purchases
- revenue
- day-by-day time series

In [0]:
from pyspark.sql import functions as F

fact = spark.table("workspace.gold.fact_interactions")

gold_daily_trends = (fact.groupBy("event_date")
    .agg(
        F.count("*").alias("events"),
        F.sum("is_view").alias("views"),
        F.sum("is_like").alias("likes"),
        F.sum("is_purchase").alias("purchases"),
        F.round(F.sum("derived_revenue"), 2).alias("revenue")
    )
    .orderBy("event_date")
)

(gold_daily_trends.write.format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("workspace.gold.daily_trends"))


In [0]:
%sql
SELECT * FROM workspace.gold.daily_trends ORDER BY event_date;


event_date,events,views,likes,purchases,revenue
2022-12-28,18,9,4,5,64.75
2022-12-29,18,6,10,2,28.59
2022-12-30,18,2,10,6,53.23
2022-12-31,18,4,2,12,553.8
2023-01-01,36,13,14,7,105.17
2023-01-02,18,5,10,2,85.94
2023-01-03,18,5,5,7,81.55
2023-01-04,18,4,7,5,66.21
2023-01-05,18,2,7,8,147.96
2023-01-06,17,7,4,5,166.83


### 4.4 Build gold dow_trend
date of the week trend table

In [0]:
from pyspark.sql import functions as F

fact = spark.table("workspace.gold.fact_interactions")

dow_trends = (
    fact
    # 1) derive day of week
    .withColumn("dow_num", F.dayofweek(F.col("event_date")))  # 1=Sun ... 7=Sat
    .withColumn(
        "day_of_week",
        F.when(F.col("dow_num") == 1, "Sun")
         .when(F.col("dow_num") == 2, "Mon")
         .when(F.col("dow_num") == 3, "Tue")
         .when(F.col("dow_num") == 4, "Wed")
         .when(F.col("dow_num") == 5, "Thu")
         .when(F.col("dow_num") == 6, "Fri")
         .when(F.col("dow_num") == 7, "Sat")
    )

    # 2) aggregate
    .groupBy("dow_num", "day_of_week")
    .agg(
        F.count("*").alias("events"),
        F.sum("is_view").alias("views"),
        F.sum("is_like").alias("likes"),
        F.sum("is_purchase").alias("purchases"),
        F.round(F.sum("derived_revenue"), 2).alias("revenue"),   # ⭐ round to 2 decimals
        F.countDistinct("event_date").alias("active_days")
    )

    # 3) normalized metric for dashboard
    .withColumn(
        "avg_events_per_day",
        F.round(F.col("events") / F.col("active_days"), 2)
    )

    # 4) ordering & cleanup
    .orderBy("dow_num")
    .drop("dow_num")
)

# 5) write Gold table
(dow_trends.write.format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("workspace.gold.dow_trends"))


In [0]:
%sql
SELECT * FROM workspace.gold.dow_trends;


day_of_week,events,views,likes,purchases,revenue,active_days,avg_events_per_day
Sun,434,123,161,131,2927.85,52,8.35
Mon,415,129,156,112,4155.19,51,8.14
Tue,436,121,174,120,3254.43,52,8.38
Wed,440,128,160,133,544504.24,53,8.3
Thu,440,133,163,126,4590.81,53,8.3
Fri,417,120,168,113,3402.98,52,8.02
Sat,417,117,163,120,3086.93,52,8.02


### 4.5 Build category kpi

In [0]:
from pyspark.sql import functions as F

fact = spark.table("workspace.gold.fact_interactions")

gold_cetegory_clean = (fact
    .withColumn("category_lvl1", F.trim(F.split(F.col("category"), "\\|")[0]))
    .withColumn("category_lvl2", F.trim(F.split(F.col("category"), "\\|")[1]))
)

(gold_cetegory_clean.write.format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("workspace.gold.category"))


In [0]:
%sql
SELECT * FROM workspace.default.gold_fact_interactions
LIMIT 50;


user_id,product_id,interaction_type,event_ts,event_date,product_name,category,selling_price,age,gender,location,subscription_status,is_view,is_like,is_purchase,derived_revenue,category_lvl1,category_lvl2
681,ae52ba51dba91bcbe1b2a4815903c40a,like,2023-01-09T10:00:00.000Z,2023-01-09,"Nino Percussion NINO-NS355 14-Inch Cymbal with Holding Straps, Nickel Silver",Toys & Games | Learning & Education | Musical Instruments | Drums & Percussion,52.69,58,Male,Tennessee,true,0,1,0,0.0,Toys & Games,Learning & Education
2413,12658e8bae58486bfa2a2885380c07cb,view,2023-12-07T08:00:00.000Z,2023-12-07,Disney Mickey Popcorn Coral Fleece Blanket,Home & Kitchen | Bedding | Kids' Bedding | Blankets & Throws,20.02,53,Male,Oregon,false,1,0,0,0.0,Home & Kitchen,Bedding
2806,88ea1baa294283526e838357de9dd110,view,2023-09-15T10:00:00.000Z,2023-09-15,Marvel Spider-Man Titan Hero Series Villains Carnage Figure,null,15.08,18,Female,Nevada,false,1,0,0,0.0,null,null
2632,165e609717373146f50529986ded897d,like,2023-03-25T10:00:00.000Z,2023-03-25,"Funko 289 Animation: Pop! Rocky & Bullwinkle Collectors Magician Bullwinkle, Flying Rocky, Bullwinkle Fearlessleader, Set",Toys & Games | Toy Figures & Playsets | Action Figures,6.89,53,Male,Connecticut,false,0,1,0,0.0,Toys & Games,Toy Figures & Playsets
2873,87f78a1ba957dde09300521b7320374e,like,2023-12-02T08:00:00.000Z,2023-12-02,"Redcat Racing 02075 Steering Bell Crank, 1Piece",Hobbies | Remote & App Controlled Vehicles & Parts | Remote & App Controlled Vehicle Parts | Power Plant & Driveline Systems | Suspension Systems & Parts,6.79,63,Female,Connecticut,false,0,1,0,0.0,Hobbies,Remote & App Controlled Vehicles & Parts
2040,01bacf123137341fc0b1e86aec29c3e7,like,2023-11-22T08:00:00.000Z,2023-11-22,Fisher-Price Laugh & Learn Sweet Manners Tea Set,null,19.88,35,Male,Minnesota,false,0,1,0,0.0,null,null
2424,75d64da1aa9c1f6fcd9d0a2bf20a71dd,like,2023-12-18T08:00:00.000Z,2023-12-18,"School Smart Blunt Tip Student Scissors, 6-1/4 Inches, Pack of 12",Office Products | Office & School Supplies | Education & Crafts | Students Scissors,25.32,49,Male,West Virginia,false,0,1,0,0.0,Office Products,Office & School Supplies
2430,95e0cd18ad6c6379fd03351480e806b3,purchase,2023-12-24T08:00:00.000Z,2023-12-24,RepliCade Insert Coin Keychain,Toys & Games | Hobbies | Coin Collecting,14.99,57,Male,Virginia,false,0,0,1,14.99,Toys & Games,Hobbies
1293,060c4fbc66821a1268e65a65dd877f36,like,2023-10-30T08:00:00.000Z,2023-10-30,"Modular Robotics Cubelets Robot Blocks - Code & Construct Educator Pack - STEM Education & Coding Robot, Free Lesson Plans, Ages 4+",Toys & Games | Learning & Education | Science Kits & Toys,1734.00,26,Male,Kentucky,false,0,1,0,0.0,Toys & Games,Learning & Education
550,0b875d54bbfc8902a6f053db9b395052,view,2023-12-20T08:00:00.000Z,2023-12-20,Rhode Island Novelty Monkey Bean Filled Plush Stuffed Animal,Home & Kitchen | Furniture | Kids' Furniture,9.99,31,Male,Arizona,true,1,0,0,0.0,Home & Kitchen,Furniture


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

fact = spark.table("workspace.gold.fact_interactions")

# 1) 基础聚合：每个 category 一行
category_kpis = (
    fact
    .groupBy("category")
    .agg(
        F.count("*").alias("events"),
        F.sum("is_purchase").alias("purchases"),
        F.round(F.sum("derived_revenue"), 2).alias("revenue"),
        F.countDistinct(
            F.when(F.col("is_purchase") == 1, F.col("user_id"))
        ).alias("buyers"),
        F.countDistinct("event_date").alias("active_days")
    )
)

# 2) 计算 revenue share（占比）
total_revenue = category_kpis.agg(F.sum("revenue").alias("total")).collect()[0]["total"]

category_kpis = (
    category_kpis
    .withColumn(
        "revenue_share",
        F.round(F.col("revenue") / F.lit(total_revenue), 4)
    )
    .orderBy(F.col("revenue").desc())
)

# 3) 写入 Gold 表
(category_kpis.write.format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("workspace.gold.category_kpis"))


In [0]:
%sql
SELECT * FROM workspace.gold.category_kpis
LIMIT 50;


category,events,purchases,revenue,buyers,active_days,revenue_share
Toys & Games | Dress Up & Pretend Play | Pretend Play,35,10,540277.21,10,31,0.9547
null,275,84,1693.35,84,169,0.003
Toys & Games | Sports & Outdoor Play | Play Sets & Playground Equipment | Play & Swing Sets,6,1,1265.0,1,5,0.0022
Toys & Games | Hobbies | Remote & App Controlled Vehicles & Parts | Remote & App Controlled Vehicles | Quadcopters & Multirotors,6,4,917.94,4,6,0.0016
Toys & Games | Stuffed Animals & Plush Toys | Stuffed Animals & Teddy Bears,78,24,738.62,24,68,0.0013
"Sports & Outdoors | Outdoor Recreation | Skates, Skateboards & Scooters | Skateboarding | Standard Skateboards & Longboards | Longboards",7,4,736.62,4,7,0.0013
Toys & Games | Games & Accessories | Board Games,113,33,717.19,33,90,0.0013
"Toys & Games | Tricycles, Scooters & Wagons | Ride-On Toys & Accessories",10,3,658.46,3,10,0.0012
Sports & Outdoors | Outdoor Recreation | Cycling | Kids' Bikes & Accessories | Kids' Bikes,8,6,606.85,6,8,0.0011
Toys & Games | Hobbies | Remote & App Controlled Vehicles & Parts | Remote & App Controlled Vehicles | Trucks,3,2,581.99,2,3,0.001


In [0]:
from pyspark.sql import functions as F

fact = spark.table("workspace.gold.fact_interactions")

base = (
    fact
    .filter(F.col("category").isNotNull() & (F.trim(F.col("category")) != ""))
    .withColumn("parts", F.split(F.col("category"), "\\|"))
    .withColumn("category_lvl1", F.trim(F.col("parts")[0]))
    .drop("parts")
    .filter(F.col("category_lvl1").isNotNull() & (F.col("category_lvl1") != ""))
)

lvl1_kpis = (
    base
    .groupBy("category_lvl1")
    .agg(
        F.count("*").alias("events"),
        F.sum("is_view").alias("views"),
        F.sum("is_like").alias("likes"),
        F.sum("is_purchase").alias("purchases"),
        F.round(F.sum("derived_revenue"), 2).alias("revenue"),
        F.countDistinct("event_date").alias("active_days"),
        F.countDistinct(F.when(F.col("is_purchase") == 1, F.col("user_id"))).alias("buyers"),
    )
    .withColumn("avg_events_per_day", F.round(F.col("events") / F.col("active_days"), 2))
)

# revenue_share（占比）
total_rev = lvl1_kpis.agg(F.sum("revenue").alias("total")).collect()[0]["total"]
lvl1_kpis = lvl1_kpis.withColumn(
    "revenue_share",
    F.when(F.lit(total_rev) > 0, F.round(F.col("revenue") / F.lit(total_rev), 4)).otherwise(F.lit(None))
).orderBy(F.col("revenue").desc())

(lvl1_kpis.write.format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("workspace.gold.category_lvl1_kpis"))


In [0]:
from pyspark.sql import functions as F

fact = spark.table("workspace.gold.fact_interactions")

base = (
    fact
    .filter(F.col("category").isNotNull() & (F.trim(F.col("category")) != ""))
    .withColumn("parts", F.split(F.col("category"), "\\|"))
    .withColumn("category_lvl1", F.trim(F.col("parts")[0]))
    .withColumn("category_leaf", F.trim(F.element_at(F.col("parts"), -1)))  # last element
    .drop("parts")
    .filter(F.col("category_lvl1").isNotNull() & (F.col("category_lvl1") != ""))
    .filter(F.col("category_leaf").isNotNull() & (F.col("category_leaf") != ""))
)

leaf_kpis = (
    base
    .groupBy("category_lvl1", "category_leaf")
    .agg(
        F.count("*").alias("events"),
        F.sum("is_view").alias("views"),
        F.sum("is_like").alias("likes"),
        F.sum("is_purchase").alias("purchases"),
        F.round(F.sum("derived_revenue"), 2).alias("revenue"),
        F.countDistinct("event_date").alias("active_days"),
        F.countDistinct(F.when(F.col("is_purchase") == 1, F.col("user_id"))).alias("buyers"),
    )
    .withColumn("avg_events_per_day", F.round(F.col("events") / F.col("active_days"), 2))
)

# revenue_share（占比）——在 leaf 级别也很有用
total_rev = leaf_kpis.agg(F.sum("revenue").alias("total")).collect()[0]["total"]
leaf_kpis = leaf_kpis.withColumn(
    "revenue_share",
    F.when(F.lit(total_rev) > 0, F.round(F.col("revenue") / F.lit(total_rev), 4)).otherwise(F.lit(None))
).orderBy(F.col("revenue").desc())

(leaf_kpis.write.format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("workspace.gold.category_leaf_kpis"))


In [0]:
%sql
SELECT * FROM workspace.gold.category_lvl1_kpis ORDER BY revenue DESC LIMIT 20;


category_lvl1,events,views,likes,purchases,revenue,active_days,buyers,avg_events_per_day,revenue_share
Toys & Games,2098,595,813,594,557390.6,356,594,5.89,0.9879
Sports & Outdoors,119,31,48,36,2827.64,97,36,1.23,0.005
Home & Kitchen,163,56,54,44,1887.82,105,44,1.55,0.0033
"Clothing, Shoes & Jewelry",176,58,69,46,968.56,128,46,1.38,0.0017
Baby Products,44,15,13,14,292.98,40,14,1.1,5.0E-4
"Patio, Lawn & Garden",5,2,0,3,226.57,5,3,1.0,4.0E-4
Hobbies,6,0,3,3,213.86,6,3,1.0,4.0E-4
Office Products,28,11,6,11,147.07,26,11,1.08,3.0E-4
"Arts, Crafts & Sewing",44,17,18,9,102.31,43,9,1.02,2.0E-4
Health & Household,7,1,1,5,50.39,7,5,1.0,1.0E-4


In [0]:
%sql
SELECT * FROM workspace.gold.category_leaf_kpis ORDER BY revenue DESC LIMIT 50;

category_lvl1,category_leaf,events,views,likes,purchases,revenue,active_days,buyers,avg_events_per_day,revenue_share
Toys & Games,Pretend Play,35,10,14,10,540277.21,31,10,1.13,0.9575
Toys & Games,Play & Swing Sets,6,1,3,1,1265.0,5,1,1.2,0.0022
Toys & Games,Quadcopters & Multirotors,6,1,1,4,917.94,6,4,1.0,0.0016
Toys & Games,Stuffed Animals & Teddy Bears,78,22,29,24,738.62,68,24,1.15,0.0013
Sports & Outdoors,Longboards,7,1,2,4,736.62,7,4,1.0,0.0013
Toys & Games,Board Games,113,32,42,33,717.19,90,33,1.26,0.0013
Toys & Games,Ride-On Toys & Accessories,10,0,6,3,658.46,10,3,1.0,0.0012
Sports & Outdoors,Kids' Bikes,8,0,2,6,606.85,8,6,1.0,0.0011
Toys & Games,Trucks,3,0,1,2,581.99,3,2,1.0,0.001
Toys & Games,Building Toys,15,2,7,6,561.76,15,6,1.0,0.001


> ### 4.5 Build user 画像
使用那个user table

In [0]:
from pyspark.sql import functions as F

fact = spark.table("workspace.gold.fact_interactions")

user_profile_kpis = (
    fact
    .groupBy("user_id")
    .agg(
        F.count("*").alias("events"),
        F.sum("is_view").alias("views"),
        F.sum("is_like").alias("likes"),
        F.sum("is_purchase").alias("purchases"),
        F.round(F.sum(F.coalesce(F.col("derived_revenue"), F.lit(0.0))), 2).alias("revenue"),
        F.countDistinct("event_date").alias("active_days"),
        F.max("event_date").alias("last_active_date")
    )
    .withColumn("is_buyer", (F.col("purchases") > 0).cast("boolean"))
    .withColumn("avg_events_per_day", F.round(F.col("events") / F.col("active_days"), 2))
    .withColumn("avg_revenue_per_purchase",
                F.when(F.col("purchases") > 0, F.round(F.col("revenue") / F.col("purchases"), 2))
                 .otherwise(F.lit(None)))
)

(user_profile_kpis.write.format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("workspace.gold.user_profile_kpis"))


In [0]:
from pyspark.sql import functions as F

profiles = spark.table("workspace.gold.user_profile_kpis")
customers = spark.table("workspace.silver.customers").select("user_id", "age", "gender")

user_profile_enriched = (
    profiles.alias("p")
    .join(customers.alias("c"), on="user_id", how="left")
    .select("user_id", "age", "gender",
            "events", "views", "likes", "purchases", "revenue",
            "active_days", "last_active_date", "is_buyer",
            "avg_events_per_day", "avg_revenue_per_purchase")
)

(user_profile_enriched.write.format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("workspace.gold.user_profile_enriched"))


In [0]:
%sql
-- compare buyer VS non-buyer
SELECT
  is_buyer,
  COUNT(*) AS users,
  AVG(events) AS avg_events,
  AVG(purchases) AS avg_purchases,
  ROUND(AVG(revenue), 2) AS avg_revenue
FROM workspace.gold.user_profile_kpis
GROUP BY is_buyer;


is_buyer,users,avg_events,avg_purchases,avg_revenue
true,855,1.0,1.0,661.9
false,2016,1.0,0.0,0.0
null,128,1.0,null,0.0


In [0]:
%sql
-- compare gender
SELECT
  gender,
  COUNT(*) AS users,
  ROUND(AVG(events), 2) AS avg_events,
  ROUND(AVG(purchases), 2) AS avg_purchases,
  ROUND(AVG(revenue), 2) AS avg_revenue
FROM workspace.gold.user_profile_enriched
GROUP BY gender
ORDER BY users DESC;


gender,users,avg_events,avg_purchases,avg_revenue
Male,2652,1.0,0.3,212.51
Female,347,1.0,0.3,6.75


In [0]:
print("✅ Ecommerce ETL pipeline completed successfully.")


✅ Ecommerce ETL pipeline completed successfully.
